<a href="https://colab.research.google.com/github/ZhihaoDC/TFG/blob/main/marvel_social_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup Environment

In [2]:
from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/My Drive/TFG

!git pull https://github.com/ZhihaoDC/TFG

Mounted at /gdrive
/gdrive/My Drive/TFG
From https://github.com/ZhihaoDC/TFG
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


#Import Libraries

In [3]:
import pandas as pd
import numpy as np
import networkx as nx

# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors
# import seaborn as sns

# import statistics
# import math
# import itertools
# import re #regular expressions

# import plotly.express as px
# import plotly.figure_factory as ff

In [ ]:
#Hello github